In [143]:
import pandas as pd
import glob
import re
file_paths = glob.glob(r'1984 - 2022 멜론 Top.100\\*.멜론TOP100가사.csv')  # 데이터 위치 확인 필요
df_list = [pd.read_csv(file_path,index_col=0) for file_path in file_paths]
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.rename(columns={'가사':'lyrics','제목':'title','장르':'genre'},inplace=True)
combined_df.drop_duplicates(ignore_index=True,inplace=True)
combined_df = combined_df[(combined_df['lyrics']!='Nan')&(combined_df['title']!='NaN') ]
combined_df.dropna(inplace=True)
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\n+', '\n',x))
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\s*\n\s*', '\n',x))
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\s*!\n*', '\n',x))
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\s*\n', '\n',x))
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\n\s*', '\n',x))
combined_df['lyrics'] = combined_df['lyrics'].str.replace('  ', ' ')
combined_df['lyrics'] = combined_df['lyrics'].str.strip()
df_1,df_2,df_3,df_4,df_5,df_6 = (combined_df.copy(deep=True) for _ in range(6))


In [ ]:
# combined_df['lyrics'] = combined_df['lyrics'].str.replace('   \n', '\n')
#combined_df['lyrics'] = combined_df['lyrics'].str.replace('\n\n\n', '\n')
# combined_df['lyrics'] = combined_df['lyrics'].str.replace('\n  \n', '\n')
# combined_df['lyrics'] = combined_df['lyrics'].str.replace('\n \n', '\n')
#combined_df['lyrics'] = combined_df['lyrics'].str.replace('\n\n', '\n')
#combined_df['lyrics'] = combined_df['lyrics'].str.replace('\n\n\n ', '\n')
# combined_df['lyrics'] = combined_df['lyrics'].str.replace('!', '')

In [144]:
#단어단위 뭉치 NL줄 단위\n",
def word_N_split(lyrics_row,l_num):
    l_num =l_num
    word_sum = 0
    lyrics_mung = []
    lyrics_word =[]
    for word in lyrics_row.split('\n'): #df_6['lyrics'].iloc[1]
        word_sum += len(word)
        lyrics_word.append(word)

        if word_sum>=l_num:
            lyrics_mung.append(lyrics_word)
            word_sum = 0
            lyrics_word=[]
    return [[word for phrase in sublist for word in phrase.split() if word !=''] for sublist in lyrics_mung]

#한줄단위 뭉치 NL줄 단위
def line_N_split(lyrics_row,l_num):
    word_sum = 0
    lyrics_mung = []
    lyrics_word =[]
    for word in lyrics_row.split('\n'): #df_6['lyrics'].iloc[1]

        word_sum += len(word)
        lyrics_word.append(word)

        if word_sum>=l_num:
            lyrics_mung.append(lyrics_word)
            word_sum = 0
            lyrics_word=[]
    return lyrics_mung

def count_char(text):
    return sum(1 for char in text if char.isalpha())

In [145]:
# label 한syllable_strategy (필요없어짐)
#df['lyrics'].apply(lambda x: [line.split(' ') for line in x.split('\\n')] if pd.notnull(x) else x)

# label syllable_limit마다
n_ = 1
df_1['label'] = df_1['lyrics'].apply(lambda x: [' '.join(x.split('\n')[i:i+n_]).split(' ') for i in range(0, len(x.split('\n')), n_) if x.split('\n')[i:i+n_]!=''])
df_1['chunk_strategy'] = 'word'
df_1['sampling_strategy'] ='one'
df_1['syllable_limit'] = 0
df_1['mungchi'] = df_1['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])
# label 전곡
df_2['label'] = df_2['lyrics'].apply(lambda x: [word for part in x.split('\n') for word in part.split(' ') if word])
df_2['chunk_strategy'] = 'word'
df_2['sampling_strategy'] ='total'
df_2['syllable_limit'] = 0
df_2['mungchi'] = df_2['label'].apply(lambda x: [count_char(sublist) for sublist in x])

l_num = 30
df_3['label'] = df_3['lyrics'].apply(lambda x: word_N_split(x,l_num))
df_3['chunk_strategy'] = 'word'
df_3['sampling_strategy'] ='L'
df_3['syllable_limit'] = l_num
df_3['mungchi'] = df_3['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])
# 한label syllable_limit
n_ = 1
df_4['label'] = df_4['lyrics'].apply(lambda x: [x.strip().split('\n')[i:i+n_] for i in range(0, len(x.strip().split('\n')), n_)])
df_4['chunk_strategy'] = 'line'
df_4['sampling_strategy'] ='one'
df_4['syllable_limit'] = 0
df_4['mungchi'] = df_4['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])
# 한label 전곡
df_5['label']= df_5['lyrics'].apply(lambda x: [x.strip().split('\n')])
df_5['chunk_strategy'] = 'line'
df_5['sampling_strategy'] ='total'
df_5['syllable_limit'] = 0
df_5['mungchi'] = df_5['label'].apply(lambda x: [[count_char(word.strip()) for word in sublist] for sublist in x])

df_6['label'] = df_6['lyrics'].apply(lambda x: line_N_split(x,l_num))
df_6['chunk_strategy'] = 'line'
df_6['sampling_strategy'] ='L'
df_6['syllable_limit'] = l_num
df_6['mungchi'] = df_6['label'].apply(lambda x: [[count_char(word) for word in sublist] for sublist in x])

In [146]:
dataset = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], axis=0)
dataset = dataset[['title','lyrics','genre','mungchi','label','chunk_strategy','sampling_strategy','syllable_limit']]

In [147]:
# data 저장
dataset.to_csv('lyrics_dataset.csv',index_label=False)
dataset.reset_index().to_json('lyrics_dataset.json')

In [148]:
df = dataset
df.loc[(df['title']=='J에게') & (df['chunk_strategy']=='line') & (df['sampling_strategy']=='one')]['mungchi'][0]

[[15],
 [14],
 [14],
 [13],
 [17],
 [14],
 [14],
 [14],
 [14],
 [17],
 [14],
 [14],
 [14],
 [14],
 [7]]

In [149]:
df.loc[(df['title']=='J에게')]['mungchi']

0    [[1, 3, 3, 1, 2, 2, 3], [1, 3, 3, 2, 5], [1, 3...
0    [1, 3, 3, 1, 2, 2, 3, 1, 3, 3, 2, 5, 1, 3, 3, ...
0    [[1, 3, 3, 1, 2, 2, 3, 1, 3, 3, 2, 5], [1, 3, ...
0    [[15], [14], [14], [13], [17], [14], [14], [14...
0    [[15, 14, 14, 13, 17, 14, 14, 14, 14, 17, 14, ...
0    [[15, 14], [14, 13], [17, 14], [14, 14], [14, ...
Name: mungchi, dtype: object

In [37]:
# 리스트안에 빈칸 있는 가사 찾기
def zero_check(df):
    idx = 0
    for i in df.label:
        idx+=1
        for j in i:
            for x in j:
                if len(x)==0:
                    print(idx-1,len(x),i)
    return

# 리스트안에 특정문자 있는 가사 찾기
def word_check(df,word):
    idx = 0
    for i in df.label:
        idx+=1
        for j in i:
            for x in j:
                if x==word:
                    print(idx-1,i)
    return
                

In [150]:
zero_check(df_1,)

In [ ]:
idx = 0
for i in df_1.lyrics:
    idx+=1
    for j in i:
        print(j)

In [136]:
df_1.iloc[73]

title                                                               그대
가수                                                                 이태원
lyrics               그대 아름다운 얼굴에\n슬픈 미소 짓지 말아요\n그대 사랑하는 이 마음 언제라도 있...
genre                                                           포크/블루스
label                [[그대, 아름다운, 얼굴에], [슬픈, 미소, 짓지, 말아요], [그대, 사랑하는...
chunk_strategy                                                    word
sampling_strategy                                                  one
syllable_limit                                                       0
mungchi              [[2, 4, 3], [2, 2, 2, 3], [2, 4, 1, 2, 4, 3], ...
Name: 83, dtype: object

In [45]:
dataset['lyrics'] = dataset['lyrics'].str.replace('\n\n', '\n', regex=False)

In [66]:
print(df_1[df_1['title'] ==' 비내리는 영동교']['lyrics'].str.replace(r'\n+', '\n').tolist())
print(df_1[df_1['title'] =='낭만고양이']['lyrics'].tolist())

['sweet little kitty\nsweet little kitty -\n\n내 두눈 밤이면 별이 되지 \n나의 집은 뒷골목 달과 별이 뜨지요\n두번 다신 생선가게 털지 않아\n서럽게 울던 날들 나는 외톨이라네\n\n이젠 바다로 떠날거예요(더 자유롭게)\n거미로 그물 쳐서 물고기 잡으러\n\n나는 낭만 고양이\n슬픈 도시를 비춰 춤추는 작은 별빛\n나는 낭만 고양이\n홀로 떠나가버린 깊고 슬픈 나의 바다여\n\n(sweet) sweet little kitty -\n\n깊은 바다 자유롭게 날던 내가\n한 없이 밑으로만 가라 앉고 있는데\n이젠 바다로 떠날 거예요(더 자유롭게)\n거미로 그물 쳐서 물고기 잡으러\n\n나는 낭만 고양이\n슬픈 도시를 비춰 춤추는 작은 별빛\n나는 낭만 고양이 \n홀로 떠나가버린 깊고 슬픈 나의 바다여\n\nsweet little kitty\nsweet little kitty -\n\n(나는 낭만 고양이 홀로 떠나가버린..)\n\n나는 낭만 고양이\n슬픈 도시를 비춰 (도시를 비춰)\n춤추는 작은 별빛\n\n나는 낭만 고양이\n홀로 떠나가버린 (떠나가 버린 아)\n깊고 슬픈 나의 바다여\n\n(sweet) sweet little kitty -']


In [137]:
#print(combined_df[combined_df['title'] =='Real Love']['lyrics'].str.replace(r'\n+', '\n').tolist())
print(combined_df[combined_df['title'] =='그대']['lyrics'].tolist())

['그대 아름다운 얼굴에\n슬픈 미소 짓지 말아요\n그대 사랑하는 이 마음 언제라도 있지요\n그대 아름다운 마음에\n슬픈 추억 갖지 말아요\n그대 좋아하는 이마음 언제라도 있지요\n우리는 누구입니까\n빈 언덕의 자운영꽃\n혼자힘으로 일어설 수 없는\n반짝이는 조약돌\n이름을 얻지못한 구석진 마을의\n투명한 시냇물\n일제히 흰 띠를 두르고 스스로 다가오는\n첫눈입니다\n우리는 무엇입니까\n늘 앞질러 사랑케 하실 힘 덜어내고도\n몇배로 다시 고이는 힘\n이파리도 되고 실팍한 줄기도 되고\n아\n 한몫에 그대를 다 품을 수 있는\n씨앗으로 남고 싶습니다\n허물없이 맨발인 넉넉한 저녁입니다\n뜨거운 목젖까지 알아내고도\n코끝으로까지 발이 저린\n우리는 나무입니다\n우리는 어떤 노래입니까\n이노리나무 정수리에 낭낭 걸린 노래 한소절\n그대 아름다운 얼굴에\n슬픈 미소 짓지 말아요\n그대 사랑하는 이마음 언제라도 있지요\n아름다운 세상을 눈물나게 하는\n눈물나는 세상을 아름답게 하는\n그대와 나는 두고 두고 사랑해야 합니다\n그것이 내가 네게로 이르는 길\n네가 깨끗한 얼굴로 내게로 되돌아 오는길\n그대와 나는 내리 내리 사랑하는 일만\n남겨두어야 합니다\n그대 아름다운 마음에\n슬픈 추억갖지 말아요\n그대 좋아하는 이마음 영원토록 있지요']


In [113]:
print(re.sub(r'\n+', '\n',combined_df[combined_df['title'] =='Real Love']['lyrics']))
print(combined_df[combined_df['title'] =='Real Love']['lyrics'].tolist())

TypeError: expected string or bytes-like object

In [115]:
combined_df['lyrics'] =combined_df['lyrics'].apply(lambda x: re.sub(r'\n+', '\n',x))

In [108]:
combined_df[combined_df['title'] =='Real Love']['lyrics'].tolist()

["우린 이 음악을 빌려\n이 분위기를 빌려\n지금 이 공기 이건 마치\n7시간 비행 뒤 만난 것 같은 섬\npink빛 하늘을 빌려\nTalkin' 'bout the real love\n\n낯선 과일 향기가 퍼져\n이 기분은 뭘까\n아주 살짝 감긴 나의 눈이 빛나고 있잖아\n사랑을 말하기 이보다 완벽한 밤은 없어\n지금이야\n\n우린 이 음악을 빌려\n이 분위기를 빌려\n지금 이 공기 이건 마치\n7시간 비행 뒤 만난 것 같은 섬\npink빛 하늘을 빌려\nTalkin' 'bout the real love\n\n웅성거린 소리는 모두\n다른 외국어 같아\n너에게 1, 2, 3 최면처럼 막 빠져드는 이 순간\n사랑을 말하기 이보다 완벽한 밤은 없어\n지금이야\n\n우린 이 음악을 빌려\n이 분위기를 빌려\n지금 이 공기 이건 마치\n7시간 비행 뒤 만난 것 같은 섬\npink빛 하늘을 빌려\nTalkin' 'bout the real love\n\nReal, real love\nIt’s a Real, real love\nReal, real love\nIt’s a Real, real love\n이건 exotic experience\n우연이라기엔 영화 속 한 장면 같아 솔직히 이 모든 게\n자꾸만 느려져 천천히 가는 이 느낌\n감각은 so unreal, real love\n\n우린 이 음악을 빌려\n이 분위기를 빌려\n지금 이 공기 이건 마치\n7시간 비행 뒤 만난 것 같은 섬의\npink빛 하늘을 빌려\nTalkin' 'bout the real love"]